In [0]:
# Loading the necessary library files
import pandas as pd

In [3]:
# Loading data from the drive

# Please change the filename as per the location where the file is stored
filename = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter13/Dataset/bank-full.csv'
# Loading the data using pandas

bankData = pd.read_csv(filename,sep=";")
bankData.head()

age           job  marital  education  ... pdays  previous poutcome   y
0   58    management  married   tertiary  ...    -1         0  unknown  no
1   44    technician   single  secondary  ...    -1         0  unknown  no
2   33  entrepreneur  married  secondary  ...    -1         0  unknown  no
3   47   blue-collar  married    unknown  ...    -1         0  unknown  no
4   33       unknown   single    unknown  ...    -1         0  unknown  no

[5 rows x 17 columns]

In [0]:
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()

In [0]:
# Converting each of the columns to scaled version

bankData['ageScaled'] = rob_scaler.fit_transform(bankData['age'].values.reshape(-1,1))
bankData['balScaled'] = rob_scaler.fit_transform(bankData['balance'].values.reshape(-1,1))
bankData['durScaled'] = rob_scaler.fit_transform(bankData['duration'].values.reshape(-1,1))

In [0]:
# Dropping the original columns

bankData.drop(['age','balance','duration'], axis=1, inplace=True)

In [0]:
# Converting all the categorical variables to dummy variables
bankCat = pd.get_dummies(bankData[['job','marital','education','default','housing','loan','contact','month','poutcome']])

In [8]:
# Seperating the numerical data
bankNum = bankData[['ageScaled','balScaled','day','durScaled','campaign','pdays','previous']]
bankNum.shape

(45211, 7)

In [9]:
# Merging with the original data frame
# Preparing the X variables
X = pd.concat([bankCat, bankNum], axis=1)
print(X.shape)
# Preparing the Y variable
Y = bankData['y']
print(Y.shape)
X.head()

(45211, 51)
(45211,)


job_admin.  job_blue-collar  job_entrepreneur  ...  campaign  pdays  previous
0           0                0                 0  ...         1     -1         0
1           0                0                 0  ...         1     -1         0
2           0                0                 1  ...         1     -1         0
3           0                1                 0  ...         1     -1         0
4           0                0                 0  ...         1     -1         0

[5 rows x 51 columns]

In [0]:
from sklearn.model_selection import train_test_split
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

In [11]:
# Let us first join the train_x and train_y for ease of operation

trainData = pd.concat([X_train,y_train],axis=1)
trainData.head()

job_admin.  job_blue-collar  job_entrepreneur  ...  pdays  previous   y
19100           1                0                 0  ...     -1         0  no
37958           1                0                 0  ...    289        19  no
12451           0                1                 0  ...     -1         0  no
18263           0                0                 0  ...     -1         0  no
5128            0                0                 0  ...     -1         0  no

[5 rows x 52 columns]

In [12]:
# Finding the indexes of the sample data set where the propensity is 'yes'
ind = trainData[trainData['y']=='yes'].index
print(len(ind))

3723


In [13]:
# Seperate the minority classes
minData = trainData.loc[ind]
print(minData.shape)

(3723, 52)


In [14]:
# Finding indexes of majority class
ind1 = trainData[trainData['y']=='no'].index
print(len(ind1))

27924


In [15]:
# Seperating the majority class
majData = trainData.loc[ind1]
print(majData.shape)
majData.head()

(27924, 52)


job_admin.  job_blue-collar  job_entrepreneur  ...  pdays  previous   y
19100           1                0                 0  ...     -1         0  no
37958           1                0                 0  ...    289        19  no
12451           0                1                 0  ...     -1         0  no
18263           0                0                 0  ...     -1         0  no
5128            0                0                 0  ...     -1         0  no

[5 rows x 52 columns]

In [0]:
# Take a random sample equal to length of the minority class to make the data set balanced

majSample = majData.sample(n=len(ind),random_state = 123)

In [17]:

print(majSample.shape)
majSample.head()



(3723, 52)


job_admin.  job_blue-collar  job_entrepreneur  ...  pdays  previous   y
17387           0                0                 0  ...     -1         0  no
34679           0                1                 0  ...    250         3  no
26572           1                0                 0  ...     -1         0  no
3280            0                0                 0  ...     -1         0  no
4434            0                0                 0  ...     -1         0  no

[5 rows x 52 columns]

In [18]:
# Concatinating both data sets and then shuffling the data set

balData = pd.concat([minData,majSample],axis = 0)
print('balanced data set shape',balData.shape)

balanced data set shape (7446, 52)


In [19]:
# Shuffling the data set

from sklearn.utils import shuffle

balData = shuffle(balData)
balData.head()

job_admin.  job_blue-collar  job_entrepreneur  ...  pdays  previous    y
44281           0                0                 0  ...    221         2  yes
7655            0                0                 0  ...     -1         0   no
43702           0                0                 0  ...     -1         0  yes
22481           0                0                 1  ...     -1         0  yes
45175           0                0                 1  ...     93         5  yes

[5 rows x 52 columns]

In [20]:
# Making the new X_train and y_train

X_trainNew = balData.iloc[:,0:51]
print(X_trainNew.head())

y_trainNew = balData['y']
print(y_trainNew.head())

       job_admin.  job_blue-collar  job_entrepreneur  ...  campaign  pdays  previous
44281           0                0                 0  ...         1    221         2
7655            0                0                 0  ...         2     -1         0
43702           0                0                 0  ...         1     -1         0
22481           0                0                 1  ...         6     -1         0
45175           0                0                 1  ...         3     93         5

[5 rows x 51 columns]
44281    yes
7655      no
43702    yes
22481    yes
45175    yes
Name: y, dtype: object


In [21]:
from sklearn.linear_model import LogisticRegression
# Defining the LogisticRegression function
bankModel1 = LogisticRegression()
bankModel1.fit(X_trainNew, y_trainNew)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
# Predicting on the test
pred = bankModel1.predict(X_test)
print('Accuracy of Logistic regression model prediction on test set for balanced data set: {:.2f}'.format(bankModel1.score(X_test, y_test)))



Accuracy of Logistic regression model prediction on test set for balanced data set: 0.85


In [23]:
# Confusion Matrix for the model
from sklearn.metrics import confusion_matrix
confusionMatrix = confusion_matrix(y_test, pred)
print(confusionMatrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

[[10204  1794]
 [  289  1277]]
              precision    recall  f1-score   support

          no       0.97      0.85      0.91     11998
         yes       0.42      0.82      0.55      1566

    accuracy                           0.85     13564
   macro avg       0.69      0.83      0.73     13564
weighted avg       0.91      0.85      0.87     13564

